In [14]:
import pandas as pd
df = pd.read_csv("PDF_DATA\\pre_train.csv", index_col=0)
df

,/JS,/OpenAction,/AcroForm,/Annot,/Length,/Filter,/FlateDecode,/XObject,/CreationDate,/Author,...,eval,replace,fromCharCode,getAnnots,ip,xref_size,trailer_size,file_size,nof_stream,class
file_001,0.666667,1.0,NaN,1.0,NaN,0.666667,0.666667,0.0,0.0,0.0,...,NaN,NaN,1.0,1.0,0.0,0.333333,0.333333,0.074150,0.875,1.0
file_002,0.666667,1.0,NaN,1.0,NaN,0.666667,0.666667,0.0,0.0,0.0,...,NaN,NaN,1.0,1.0,0.0,0.333333,0.333333,0.033158,1.000,1.0
file_003,0.666667,1.0,NaN,1.0,NaN,0.666667,0.666667,0.0,0.0,0.0,...,NaN,NaN,1.0,1.0,0.0,0.333333,0.333333,0.033736,1.000,1.0
file_011,0.666667,1.0,NaN,1.0,NaN,0.666667,0.666667,0.0,0.0,0.0,...,NaN,NaN,1.0,1.0,0.0,0.333333,0.333333,0.037542,1.000,1.0
file_012,0.666667,1.0,NaN,1.0,NaN,0.666667,0.666667,0.0,0.0,0.0,...,NaN,NaN,1.0,1.0,0.0,0.333333,0.333333,0.082498,0.875,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
file_5336,0.666667,1.0,NaN,1.0,NaN,0.666667,0.666667,0.0,0.0,0.0,...,NaN,NaN,1.0,1.0,0.0,0.333333,0.333333,0.033395,1.000,1.0
file_5337,0.000000,0.0,NaN,0.0,NaN,0.666667,0.333333,1.0,1.0,1.0,...,NaN,NaN,0.0,0.0,0.0,0.400000,0.500000,0.143468,0.375,0.0
file_5341,0.666667,1.0,NaN,1.0,NaN,0.666667,0.666667,0.0,0.0,0.0,...,NaN,NaN,1.0,1.0,0.0,0.333333,0.333333,0.034405,1.000,1.0
file_5342,0.000000,0.0,NaN,0.0,NaN,0.666667,0.666667,1.0,1.0,1.0,...,NaN,NaN,0.0,0.0,0.0,0.400000,0.500000,0.145757,0.500,0.0


#model import
- n_estimators : 학습 모델의 수, 많아질수록 성능 향상의 가능성이 있으나, 속도가 느려짐
- learning_rate : 학습률, 너무 크면 gradient 발산의 가능성이 있으며, 너무 작으면 학습이 느림
- max_depth : 최대 탐색 깊이, 너무 크면 과적합의 가능성, 너무 작으면 학습 성능 저하
- min_samples_split : 분할 종료 최소 샘플 수, 큰 수면 과적합을 막지만 학습 성능 저하 가능성
- min_samples_leaf : leaf node가 되기 위한 최소 샘플 수, min_samples_split과 비슷한 용도
- random_state : 특정 숫자로 고정 시에 실행 시마다 고정된 결과 반환

In [15]:
from xgboost import XGBClassifier
model = XGBClassifier(n_estimators=500, learning_rate=0.2, max_depth=4, random_state = 32)

In [21]:
X = df.iloc[:, :-1]
y = df.iloc[:, -1]

In [17]:
print(X)
print(y)

                /JS  /OpenAction  /AcroForm  /Annot  /Length   /Filter  \
file_001   0.666667          1.0        NaN     1.0      NaN  0.666667   
file_002   0.666667          1.0        NaN     1.0      NaN  0.666667   
file_003   0.666667          1.0        NaN     1.0      NaN  0.666667   
file_011   0.666667          1.0        NaN     1.0      NaN  0.666667   
file_012   0.666667          1.0        NaN     1.0      NaN  0.666667   
...             ...          ...        ...     ...      ...       ...   
file_5336  0.666667          1.0        NaN     1.0      NaN  0.666667   
file_5337  0.000000          0.0        NaN     0.0      NaN  0.666667   
file_5341  0.666667          1.0        NaN     1.0      NaN  0.666667   
file_5342  0.000000          0.0        NaN     0.0      NaN  0.666667   
file_5343  0.666667          1.0        NaN     1.0      NaN  0.666667   

           /FlateDecode  /XObject  /CreationDate  /Author  ...  temptmp  eval  \
file_001       0.666667       

In [18]:
# 모델 학습
model.fit(X, y)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.2, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=4, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=500, n_jobs=None,
              num_parallel_tree=None, random_state=32, ...)

In [22]:
from sklearn.metrics import accuracy_score
y_pred = model.predict(X)

In [20]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
def get_clf_eval(y_test, y_pred):
    confusion = confusion_matrix(y_test, y_pred)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    F1 = f1_score(y_test, y_pred)
    AUC = roc_auc_score(y_test, y_pred)
    print('오차행렬:\n', confusion)
    print('\n정확도: {:.4f}'.format(accuracy))
    print('정밀도: {:.4f}'.format(precision))
    print('재현율: {:.4f}'.format(recall))
    print('F1: {:.4f}'.format(F1))
    print('AUC: {:.4f}'.format(AUC))


In [23]:
get_clf_eval(y, y_pred)

오차행렬:
 [[ 481    0]
 [   0 1710]]

정확도: 1.0000
정밀도: 1.0000
재현율: 1.0000
F1: 1.0000
AUC: 1.0000


# MLP를 활용한 학습